## 1. Libraries

In [6]:
import os
import time
import numpy as np
import tensorflow as tf
from keras.datasets.mnist import load_data
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import Adam
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import array_to_img
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import Flatten
from keras.layers import Reshape
from keras.layers import Dropout
from keras.layers import LeakyReLU
from keras.utils.vis_utils import plot_model
from numpy import expand_dims
from numpy import ones
from numpy import zeros
from numpy.random import randn
from numpy.random import rand
from numpy.random import randint
from tensorflow.keras.layers import Flatten, BatchNormalization
from tensorflow.keras.layers import Activation, ZeroPadding2D
from tensorflow.keras.layers import UpSampling2D, Conv2D
from PIL import Image

## 2. Imports and Configurations

In [7]:
# Generation resolution factor 
GENERATE_RES = 3 # (1=32, 2=64, 3=96, 4=128, etc.)

# rows/cols (should be square)
GENERATE_SQUARE = 32 * GENERATE_RES 
IMAGE_CHANNELS = 3

# Preview image 
PREVIEW_ROWS = 4
PREVIEW_COLS = 7
PREVIEW_MARGIN = 16

# Size vector to generate images from
SEED_SIZE = 100

# Configuration
DATA_FOLDER = r"C:\Users\Alberto Parenti\Downloads\STUDY\NOVA IMS\DEEP LEARNING\PROJECT\crop_part1 (extract.me)\crop_part1"
EPOCHS = 50
BATCH_SIZE = 32
BUFFER_SIZE = 60000

# Nicely formatted time string
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return "{}:{:>02}:{:>05.2f}".format(h, m, s)

print(f"Real Faces for Training: {len(os.listdir(os.path.join(DATA_FOLDER)))}")

Real Faces for Training: 6430


In [8]:
#We need a way to convert our training set into the correct format
def load_real_samples(image_path):

    #Defining master array
    master_list = list()

    for image in os.listdir(image_path):
        #loading image
        img = load_img(os.path.join(image_path,image), grayscale=True)

        # convert to numpy array
        img_array = img_to_array(img)
        #Standardizing to float
        img_array = img_array.astype("float32")
        #Grtting value between 0 and 1
        img_array/=255.0

        master_list.append(img_array)

    return np.array(master_list)

## 3. Discriminator

In [9]:
#Defining the descriminator model
def define_discriminator(image_shape):
    model = Sequential()

    model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=image_shape, 
                     padding="same"))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Dropout(0.25))
    model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
    model.add(ZeroPadding2D(padding=((0,1),(0,1))))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Dropout(0.25))
    model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Dropout(0.25))
    model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Dropout(0.25))
    model.add(Conv2D(512, kernel_size=3, strides=1, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))

    return model

In [10]:
#calling the function to define the model
model = define_discriminator(image_shape=(200,200,1))

#summarizing the model
model.summary()

#plotting the model
plot_model(model, to_file = "discriminator_plot.png", show_shapes = True, show_layer_names = True)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_5 (Conv2D)           (None, 100, 100, 32)      320       
                                                                 
 leaky_re_lu_5 (LeakyReLU)   (None, 100, 100, 32)      0         
                                                                 
 dropout_5 (Dropout)         (None, 100, 100, 32)      0         
                                                                 
 conv2d_6 (Conv2D)           (None, 50, 50, 64)        18496     
                                                                 
 zero_padding2d_1 (ZeroPaddi  (None, 51, 51, 64)       0         
 ng2D)                                                           
                                                                 
 batch_normalization_4 (Batc  (None, 51, 51, 64)       256       
 hNormalization)                                      

## 4. Generator

In [11]:
# When we are training the model we need to provide it with both real and generated characters
# So we need to define a function to provide it with a random subset of real images

def generate_real_samples(dataset, n_samples):
    # choose random instances
    ix = randint(0, dataset.shape[0], n_samples)
    # retrieve selected images
    X = dataset[ix]
    # generate 'real' class labels (1)
    y = ones((n_samples, 1))
    return X, y
    

In [12]:
# generate n fake samples with class labels as for the time being we don't have a generator model
def generate_fake_samples(n_samples):
	# generate uniform random numbers in [0,1]
	X = rand(200 * 200 * n_samples)
	# reshape into a batch of grayscale images
	X = X.reshape((n_samples, 200, 200, 1))
	# generate 'fake' class labels (0)
	y = zeros((n_samples, 1))
	return X, y

In [13]:
training_data = load_real_samples(r"C:\Users\Alberto Parenti\Downloads\STUDY\NOVA IMS\DEEP LEARNING\PROJECT\crop_part1 (extract.me)\crop_part1")
training_data

C:\Users\Alberto Parenti\anaconda3\envs\datamining_practical\lib\site-packages\keras_preprocessing\image\utils.py:107: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '


array([[[[0.87058824],
         [0.92156863],
         [0.9254902 ],
         ...,
         [0.9098039 ],
         [0.88235295],
         [0.8862745 ]],

        [[0.90588236],
         [0.9372549 ],
         [0.91764706],
         ...,
         [0.9019608 ],
         [0.8862745 ],
         [0.8862745 ]],

        [[0.9254902 ],
         [0.9372549 ],
         [0.9019608 ],
         ...,
         [0.8862745 ],
         [0.8901961 ],
         [0.88235295]],

        ...,

        [[0.1764706 ],
         [0.22352941],
         [0.24705882],
         ...,
         [0.68235296],
         [0.7176471 ],
         [0.73333335]],

        [[0.1882353 ],
         [0.18039216],
         [0.1764706 ],
         ...,
         [1.        ],
         [1.        ],
         [1.        ]],

        [[0.17254902],
         [0.12941177],
         [0.11764706],
         ...,
         [0.99215686],
         [1.        ],
         [1.        ]]],


       [[[0.16862746],
         [0.16470589],
         [0.15

In [14]:
#it generates real samples from the dataset
generate_real_samples(dataset=load_real_samples(r"C:\Users\Alberto Parenti\Downloads\STUDY\NOVA IMS\DEEP LEARNING\PROJECT\crop_part1 (extract.me)\crop_part1"), n_samples = 4000)

(array([[[[0.7764706 ],
          [0.7529412 ],
          [0.7176471 ],
          ...,
          [0.29411766],
          [0.28235295],
          [0.2627451 ]],
 
         [[0.7607843 ],
          [0.7372549 ],
          [0.7019608 ],
          ...,
          [0.2901961 ],
          [0.28235295],
          [0.26666668]],
 
         [[0.73333335],
          [0.7137255 ],
          [0.68235296],
          ...,
          [0.27450982],
          [0.27058825],
          [0.2627451 ]],
 
         ...,
 
         [[0.02352941],
          [0.02352941],
          [0.01960784],
          ...,
          [0.22352941],
          [0.22352941],
          [0.22352941]],
 
         [[0.02352941],
          [0.02352941],
          [0.01960784],
          ...,
          [0.19215687],
          [0.19215687],
          [0.19215687]],
 
         [[0.02745098],
          [0.02352941],
          [0.01568628],
          ...,
          [0.16862746],
          [0.17254902],
          [0.17254902]]],
 
 
        [

In [15]:
def build_generator(seed_size, channels):
    model = Sequential()

    model.add(Dense(4*4*256,activation="relu",input_dim=seed_size))
    model.add(Reshape((4,4,256)))

    model.add(UpSampling2D())
    model.add(Conv2D(256,kernel_size=3,padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation("relu"))

    model.add(UpSampling2D())
    model.add(Conv2D(256,kernel_size=3,padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation("relu"))
   
    # Output resolution, additional upsampling
    model.add(UpSampling2D())
    model.add(Conv2D(128,kernel_size=3,padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation("relu"))

    if GENERATE_RES>1:
      model.add(UpSampling2D(size=(GENERATE_RES,GENERATE_RES)))
      model.add(Conv2D(128,kernel_size=3,padding="same"))
      model.add(BatchNormalization(momentum=0.8))
      model.add(Activation("relu"))

    # Final CNN layer
    model.add(Conv2D(channels,kernel_size=3,padding="same"))
    model.add(Activation("tanh"))

    return model

In [16]:
generator = build_generator(SEED_SIZE, IMAGE_CHANNELS)
noise = tf.random.normal([1, SEED_SIZE])
generated_image = generator(noise, training=False)
generated_image

<tf.Tensor: shape=(1, 96, 96, 3), dtype=float32, numpy=
array([[[[ 0.00520813, -0.00457027,  0.01439242],
         [-0.00742599, -0.03526122,  0.01033687],
         [ 0.00327804, -0.04023036,  0.01260732],
         ...,
         [ 0.00138838, -0.02279679, -0.00480669],
         [ 0.00374344, -0.01639503, -0.00153476],
         [-0.00331591, -0.00572373,  0.00067873]],

        [[-0.00078958, -0.01641302,  0.01373376],
         [-0.01319755, -0.05693582,  0.02874551],
         [-0.00061102, -0.06726126,  0.02339455],
         ...,
         [-0.00648004, -0.02550304, -0.00385213],
         [-0.00372244, -0.02179736,  0.00878283],
         [-0.01093831, -0.00365467,  0.0054998 ]],

        [[-0.00046098, -0.01538005,  0.00736164],
         [-0.02332798, -0.07526054,  0.00696865],
         [-0.00876229, -0.09197112,  0.01753742],
         ...,
         [-0.01567685, -0.02872184,  0.00521487],
         [-0.0107264 , -0.01900603,  0.01602516],
         [-0.02382853, -0.00592685,  0.00780756]

In [18]:
image_shape = (GENERATE_SQUARE,GENERATE_SQUARE,IMAGE_CHANNELS)

discriminator = define_discriminator(image_shape)
decision = discriminator(generated_image)
print (decision)

tf.Tensor([[0.500049]], shape=(1, 1), dtype=float32)


## 5. Loss function

In [19]:
# This method returns a helper function to compute cross entropy loss
cross_entropy = tf.keras.losses.BinaryCrossentropy()

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

## 6. Optimization

In [20]:
'''Both the generator and discriminator use Adam and the same learning rate and momentum. This does not need to be the case. If you use a GENERATE_RES greater than 3 you may need to tune
 these learning rates, as well as other training and hyperparameters.'''
generator_optimizer = tf.keras.optimizers.Adam(1.5e-4,0.5)
discriminator_optimizer = tf.keras.optimizers.Adam(1.5e-4,0.5)

## 7. Training

In [21]:
@tf.function #  the use of `tf.function` causes the function to be "compiled"
def train_step(images):
  seed = tf.random.normal([BATCH_SIZE, SEED_SIZE])

  with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
    generated_images = generator(seed, training=True)

    real_output = discriminator(images, training=True)
    fake_output = discriminator(generated_images, training=True)

    gen_loss = generator_loss(fake_output)
    disc_loss = discriminator_loss(real_output, fake_output)
    
    gradients_of_generator = gen_tape.gradient(\
        gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(\
        disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(
        gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(
        gradients_of_discriminator, 
        discriminator.trainable_variables))
  return gen_loss,disc_loss

In [22]:
def save_images(cnt,noise):
  image_array = np.full(( 
      PREVIEW_MARGIN + (PREVIEW_ROWS * (GENERATE_SQUARE+PREVIEW_MARGIN)), 
      PREVIEW_MARGIN + (PREVIEW_COLS * (GENERATE_SQUARE+PREVIEW_MARGIN)), IMAGE_CHANNELS), 
      255, dtype=np.uint8)
  
  generated_images = generator.predict(noise)

  generated_images = 0.5 * generated_images + 0.5

  image_count = 0
  for row in range(PREVIEW_ROWS):
      for col in range(PREVIEW_COLS):
        r = row * (GENERATE_SQUARE+16) + PREVIEW_MARGIN
        c = col * (GENERATE_SQUARE+16) + PREVIEW_MARGIN
        image_array[r:r+GENERATE_SQUARE,c:c+GENERATE_SQUARE] \
            = generated_images[image_count] * 255
        image_count += 1

          
  output_path = os.path.join(DATA_FOLDER,'output')
  if not os.path.exists(output_path):
    os.makedirs(output_path)
  
  filename = os.path.join(output_path,f"train-{cnt}.png")
  im = Image.fromarray(image_array)
  im.save(filename)

In [23]:
def train(dataset, epochs):
  fixed_seed = np.random.normal(0, 1, (PREVIEW_ROWS * PREVIEW_COLS, 
                                       SEED_SIZE))
  start = time.time()

  for epoch in range(epochs):
    epoch_start = time.time()

    gen_loss_list = []
    disc_loss_list = []

    for image_batch in dataset:
      t = train_step(image_batch)
      gen_loss_list.append(t[0])
      disc_loss_list.append(t[1])

    g_loss = sum(gen_loss_list) / len(gen_loss_list)
    d_loss = sum(disc_loss_list) / len(disc_loss_list)

    epoch_elapsed = time.time()-epoch_start
    print (f'Epoch {epoch+1}, gen loss={g_loss},disc loss={d_loss},'\
           f' {hms_string(epoch_elapsed)}')
    save_images(epoch,fixed_seed)

  elapsed = time.time()-start
  print (f'Training time: {hms_string(elapsed)}')

In [24]:
# Batch and shuffle the data
train_dataset = tf.data.Dataset.from_tensor_slices(training_data).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

In [25]:
train(train_dataset, EPOCHS)

ValueError: in user code:

    File "C:\Users\ALBERT~1\AppData\Local\Temp/ipykernel_7628/624590007.py", line 8, in train_step  *
        real_output = discriminator(images, training=True)
    File "C:\Users\Alberto Parenti\anaconda3\envs\datamining_practical\lib\site-packages\keras\utils\traceback_utils.py", line 67, in error_handler  **
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\Alberto Parenti\anaconda3\envs\datamining_practical\lib\site-packages\keras\engine\input_spec.py", line 264, in assert_input_compatibility
        raise ValueError(f'Input {input_index} of layer "{layer_name}" is '

    ValueError: Input 0 of layer "sequential_3" is incompatible with the layer: expected shape=(None, 96, 96, 3), found shape=(32, 200, 200, 1)


## 8. Validation

In [ ]:
#Validation
# generate n fake samples with class labels as for the time being we don't have a generator model
def generate_fake_samples_test(n_samples):
	from keras.preprocessing.image import array_to_img

	# generate uniform random numbers in [0,1]
	X = rand(200 * 200 * n_samples)
	# reshape into a batch of grayscale images
	X = X.reshape((n_samples, 200, 200, 1))

	for array in X:
		img = array_to_img(array)
		img.show()
		break

generate_fake_samples_test(400)

## 9. Finale

In [26]:
generator.save(os.path.join(DATA_FOLDER,"face_generator.h5"))

NameError: name 'DATA_PATH' is not defined